In [ ]:
%matplotlib qt

In [ ]:
import mne
import mne_bids

from config_import import get_events, event_id

In [ ]:
input_dir = './pogs_meg'
subject = 'll_180197'
session = '220217'
task = 'task'
run = '1'

bids_root = './bids_data_test'

In [ ]:
raw = mne.io.read_raw(
    f'{input_dir}/{subject}/{session}/run{run}_raw.fif',
    allow_maxshield='yes'
)
# noisy_chs, flat_chs = mne.preprocessing.find_bad_channels_maxwell(raw)
# raw.info['bads'] = noisy_chs + flat_chs + ['MEG0623', 'MEG0241']

events = get_events(raw)

In [ ]:
bids_path = mne_bids.BIDSPath(
    subject=subject,
    session=session,
    run=run,
    task=task,
    datatype='meg',
    suffix='meg',
    extension='.fif',
    root=bids_root
)


In [ ]:
bids_path = mne_bids.BIDSPath(
    subject=subject.replace('_', ''),
    session=session,
    run=run,
    task=task,
    datatype='meg',
    suffix='meg',
    extension='.fif',
    root=bids_root
)
bids_path

## Write MEG data to BIDS

In [ ]:
mne_bids.write_raw_bids(
    raw=raw,
    bids_path=bids_path,
    events_data=events,
    event_id=event_id,
    overwrite=True,
    symlink=True,
    # anonymize={'daysback': 40000},
    verbose=True
)

In [ ]:
mne_bids.print_dir_tree(bids_root)

## Read MEG data from BIDS

In [ ]:
raw = mne_bids.read_raw_bids(bids_path=bids_path)

In [ ]:
raw.plot(start=50, duration=5)

### Recover events from the annotations

In [ ]:
events, event_id = mne.events_from_annotations(raw)
event_id

## Write fine-calibration and crosstalk files

In [ ]:
cal_fname = 'meg_conf/sss_cal_nsp_2017.dat'
ct_fname = 'meg_conf/ct_sparse_nsp_2017.fif'

mne_bids.write_meg_calibration(
    calibration=cal_fname,
    bids_path=bids_path
)
mne_bids.write_meg_crosstalk(
    fname=ct_fname,
    bids_path=bids_path
)
mne_bids.print_dir_tree(bids_root)

## Write empty-room data

In [ ]:
subject_er = 'emptyroom'
task_er = 'noise'
fname_er = f'{input_dir}/{subject}/{session}/emptyroom.fif'

raw_er = mne.io.read_raw(
    fname_er,
    allow_maxshield='yes',
    verbose=False
)
date_er = raw_er.info['meas_date']
session_er = f'{date_er.year:04}{date_er.month:02}{date_er.day:02}'

bids_path_er = mne_bids.BIDSPath(
    subject=subject_er,
    session=session_er,
    task=task_er,
    datatype='meg',
    suffix='meg',
    extension='.fif',
    root=bids_root
)
bids_path_er

In [ ]:
mne_bids.write_raw_bids(
    raw=raw_er,
    bids_path=bids_path_er,
    overwrite=True,
    symlink=True,
    # anonymize={'daysback': 40000},
    verbose=True
)
mne_bids.print_dir_tree(bids_root)

### Write experimental MEG data again, specifying the empty-room measurement we just wrote

In [ ]:
mne_bids.write_raw_bids(
    raw=raw,
    bids_path=bids_path,
    events_data=events,
    event_id=event_id,
    empty_room=bids_path_er,
    overwrite=True,
    symlink=True,
    # anonymize={'daysback': 40000},
    verbose=True
)

## Write anatomical MRI data (T1)

In [ ]:
fname_t1 =f'{input_dir}/{subject}/{session}/T1.nii.gz'

bids_path_t1 = bids_path.copy().update(
    task=None,
    run=None,
    suffix='T1w',
    extension='.nii.gz',
    datatype='anat'
)
bids_path_t1

In [ ]:
mne_bids.write_anat(
    image=fname_t1,
    bids_path=bids_path_t1,
    overwrite=True
)
mne_bids.print_dir_tree(bids_root)

In [ ]:
from nilearn.plotting import plot_anat

plot_anat(str(bids_path_t1))

### Write head->MRI transformation / fiducials

In [ ]:
fs_subjects_dir = './bids_data/derivatives/freesurfer/subjects'
fs_subject = 'sub-09'
fname_trans = f'{fs_subjects_dir}/{fs_subject}/bem/{fs_subject}-trans.fif'

landmarks = mne_bids.get_anat_landmarks(
    image=fname_t1,
    info=raw.info,
    trans=fname_trans,
    fs_subject=fs_subject,
    fs_subjects_dir=fs_subjects_dir
)

mne_bids.write_anat(
    image=fname_t1,
    bids_path=bids_path_t1,
    landmarks=landmarks,
    overwrite=True
)
mne_bids.print_dir_tree(bids_root)

In [ ]:
trans = mne_bids.get_head_mri_trans(
    bids_path=bids_path,
    t1_bids_path=bids_path_t1,
    fs_subject=fs_subject,
    fs_subjects_dir=fs_subjects_dir,
    verbose=False
)
trans

In [ ]:
mne.viz.plot_alignment(
    info=raw.info,
    trans=trans,
    subject=fs_subject,
    subjects_dir=fs_subjects_dir,
    coord_frame='mri',
    surfaces='head-dense'
)

In [ ]:
# fname_fiducials = f'{fs_subjects_dir}/{fs_subject}/bem/{fs_subject}-fiducials.fif'

# dm = mne.coreg.read_fiducials(fname_fiducials)
# dm